In [3]:
pip install langchain langchain-core langchain-community langchain-openai langgraph python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
from dotenv import load_dotenv
load_dotenv()


# os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
## Langmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

os.environ["AICORE_AUTH_URL"] = os.getenv("AICORE_AUTH_URL")
os.environ["AICORE_CLIENT_ID"] = os.getenv("AICORE_CLIENT_ID")
os.environ["AICORE_CLIENT_SECRET"] = os.getenv("AICORE_CLIENT_SECRET")
os.environ["AICORE_RESOURCE_GROUP"] = os.getenv("AICORE_RESOURCE_GROUP")
os.environ["AICORE_BASE_URL"] = os.getenv("AICORE_BASE_URL")

user=os.getenv("user")
password=os.getenv("password")
host=os.getenv("host")
database=os.getenv("database")

In [8]:
pip install sqlalchemy-hana

  Using cached sqlalchemy_hana-2.6.0-py3-none-any.whl.metadata (18 kB)
  Using cached hdbcli-2.21.31-cp36-abi3-win_amd64.whl.metadata (6.3 kB)
Using cached sqlalchemy_hana-2.6.0-py3-none-any.whl (27 kB)
Using cached hdbcli-2.21.31-cp36-abi3-win_amd64.whl (3.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_community.utilities.sql_database import SQLDatabase

connection_uri = f'hana://{user}:{password}@{host}/{database}'

db = SQLDatabase.from_uri(connection_uri)

In [10]:
from langchain_community.agent_toolkits import create_sql_agent
# from langchain_openai import ChatOpenAI
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI

LLM_DEPLOYMENT_ID = os.getenv("LLM_DEPLOYMENT_ID")

llm = ChatOpenAI(model="gpt-3.5-turbo", deployment_id=LLM_DEPLOYMENT_ID)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

C:\Users\pc\AppData\Local\Temp\ipykernel_19576\2805447668.py:7: UserWarning: WARNING! model is not default parameter.
                model was transferred to model_kwargs.
                Please confirm that model is what you intended.
  llm = ChatOpenAI(model="gpt-3.5-turbo", deployment_id=LLM_DEPLOYMENT_ID)


In [11]:
agent_executor.invoke(
    "List the total sales per country. Which country's customers spent the most?"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Invoice, Orders, Sales
Invoking: `sql_db_schema` with `{'table_names': 'Orders, Sales'}`



CREATE TABLE "Orders" (
	"OrdersId" INTEGER NOT NULL, 
	"CustomerId" INTEGER NOT NULL, 
	"OrdersDate" TIMESTAMP DEFAULT CURRENT_TIMESTAMP, 
	"BillingAddress" NVARCHAR(70), 
	"BillingCity" NVARCHAR(40), 
	"BillingState" NVARCHAR(40), 
	"BillingCountry" NVARCHAR(40), 
	"BillingPostalCode" NVARCHAR(10), 
	"Total" DECIMAL(10, 2) NOT NULL, 
	CONSTRAINT "_SYS_TREE_CS_#163943_#0_#P0" PRIMARY KEY ("OrdersId")
)

/*
3 rows from Orders table:
OrdersId	CustomerId	OrdersDate	BillingAddress	BillingCity	BillingState	BillingCountry	BillingPostalCode	Total
1	101	2024-01-15 10:00:00	123 Elm St	Springfield	IL	USA	62701	150.75
2	102	2024-02-20 11:30:00	456 Maple Ave	New York	NY	USA	10001	250.5
3	103	2024-03-05 09:15:00	789 Oak Dr	São Paulo	SP	Brazil	01000-000	75
*/


CREATE TABLE "Sales" (
	"SalesId" INTEGER NOT NULL, 
	"Custo

{'input': "List the total sales per country. Which country's customers spent the most?",
 'output': 'The total sales per country are as follows:\n- USA: $802.50\n- Australia: $801.96\n- India: $680.90\n- Brazil: $510.40\n- Japan: $492.70\n\nCustomers from the **USA** spent the most.'}